In [ ]:
#!pip install hazm

# **Load Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hazm
import string

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SpatialDropout1D, Embedding, Bidirectional, LSTM, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils import class_weight

from sklearn.metrics import classification_report
from collections import Counter

# **Load Data and EDA**

In [ ]:
data = pd.read_csv('Snappfood - Sentiment Analysis.csv', delimiter='\t', on_bad_lines='skip')
data

,Unnamed: 0,comment,label,label_id
0,NaN,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,NaN,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,NaN,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,NaN,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,NaN,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0
...,...,...,...,...
69995,NaN,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,SAD,1.0
69996,NaN,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,SAD,1.0
69997,NaN,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,HAPPY,0.0
69998,NaN,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,SAD,1.0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  520 non-null    object 
 1   comment     70000 non-null  object 
 2   label       70000 non-null  object 
 3   label_id    69480 non-null  float64
dtypes: float64(1), object(3)
memory usage: 2.1+ MB


In [ ]:
data.label_id.value_counts()

0.0    34916
1.0    34564
Name: label_id, dtype: int64

In [ ]:
data.isnull().sum()

Unnamed: 0    69480
comment           0
label             0
label_id        520
dtype: int64

In [ ]:
data = data[['comment', 'label_id']]
data

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0.0
4,شیرینی وانیلی فقط یک مدل بود.,0.0
...,...,...
69995,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,1.0
69996,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,1.0
69997,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,0.0
69998,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,1.0


In [ ]:
data.dropna(inplace=True)

<ipython-input-28-f1116dacf2bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [ ]:
data

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0.0
4,شیرینی وانیلی فقط یک مدل بود.,0.0
...,...,...
69995,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,1.0
69996,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,1.0
69997,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,0.0
69998,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,1.0


# **PreProcessing**

In [ ]:
punctuations = string.punctuation + "٬" + "،"
translator = str.maketrans('','', punctuations)
stopwords = hazm.stopwords_list()
hazm.word_tokenize(data['comment'][10])
lem = hazm.Lemmatizer()
norm =hazm.Normalizer()

In [ ]:
dataset = pd.DataFrame(columns=['Text', 'Sentiment'])

for index, row in data.iterrows():
  text = row['comment']
  text_tokenized = hazm.word_tokenize(text)
  text_lem = [lem.lemmatize(x) for x in text_tokenized]
  text_norm = [norm.normalize(x) for x in text_lem]
  clean_text = [x for x in text_norm if not x in stopwords]
  final_text = [x.translate(translator) for x in clean_text]

  dataset.loc[index] = ({
      'Text' : ' '.join(final_text),
      'Sentiment' : row['label_id']
  })

In [ ]:
dataset.Sentiment.value_counts()

0.0    34916
1.0    34564
Name: Sentiment, dtype: int64

In [ ]:
dataset['Text']

0               واقعا حیف وقت نوشتنویس سرویس دهیتون افتضاح
1        قرار بودباش ۱ ساعته برسه نیم ساعت زود موقع  دی...
2        قیمت مدل اصلا کیفیت سازگار نداره  ظاهر فریبنده...
3        بودباش درست اندازه کیفیت  امیداورم کیفیتتون با...
4                                 شیرینی وانیل مدل بودباش 
                               ...                        
69995    سلام فاکتور غذا سفارش میدم احتیاج داشتدار موضو...
69996                سایز پیتزا سفارشاتی قبلا گذشتگذر شدشو
69997    قارچ اضافه اضافه کردکن بودباش اضافه کردکن نمید...
69998         همرو ۲ ساعت تاخیر اشتباه آوردن پول رفترو هوا
69999                                     فلفل تند بودباش 
Name: Text, Length: 69480, dtype: object

In [ ]:
dataset['words_count'] = dataset['Text'].apply(lambda t: len(hazm.word_tokenize(t)))
max_len = dataset["words_count"].max()
max_len

231

In [ ]:
texts = ' '.join(dataset['Text'])
tokens = hazm.word_tokenize(texts)
counter = Counter(tokens)
min_freg = 35
filtered = [word for word, count in counter.items() if count >= min_freg]
unique_words = set(filtered)
n_words = len(unique_words)
n_words

1810

# **Tokenization Pad Sequences**

In [ ]:
X = dataset['Text']
Y = dataset['Sentiment']
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.01, random_state=4234)

In [ ]:
tokenizer = Tokenizer(num_words = n_words)
tokenizer.fit_on_texts(xtrain)

def Tokenization_padSequences(x, maxlen = max_len):
    xseq = tokenizer.texts_to_sequences(x)
    xpad = pad_sequences(xseq, padding='post', maxlen = max_len)
    return xpad

xtrain_pad = Tokenization_padSequences(xtrain)
xtest_pad = Tokenization_padSequences(xtest)

In [ ]:
xtest_pad

array([[  77,   18,  554, ...,    0,    0,    0],
       [ 128,   41,  334, ...,    0,    0,    0],
       [ 300, 1577,   93, ...,    0,    0,    0],
       ...,
       [   9,   11,    1, ...,    0,    0,    0],
       [  35,  222,  289, ...,    0,    0,    0],
       [   5,    2,  172, ...,    0,    0,    0]], dtype=int32)

In [ ]:
sequences = tokenizer.texts_to_sequences(dataset['Text'])

print(dataset['Text'][200])
print(sequences[200])

تخم مرغ ۶ عدد ۲ شکستشکن نمک قوطی استوانه انتخاب کردکن پاکتی‌ها آوردآور  دقت کردکن
[5, 428, 1010, 28, 1790, 5, 9, 165, 119, 13, 72]


#**NetWork**

In [ ]:
model = Sequential()
model.add(Embedding(n_words, 80, input_length=max_len))
model.add(Bidirectional(LSTM(256, dropout=0.2, return_sequences=True)))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(128, dropout=0.2)))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='Adam', metrics=['accuracy'], loss = 'binary_crossentropy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 231, 80)           144800    
                                                                 
 bidirectional (Bidirectiona  (None, 231, 512)         690176    
 l)                                                              
                                                                 
 spatial_dropout1d (SpatialD  (None, 231, 512)         0         
 ropout1D)                                                       
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              656384    
 nal)                                                            
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                          

In [ ]:
np.unique(ytrain)

array([0., 1.])

In [ ]:
model.fit(xtrain_pad, ytrain, epochs=20, batch_size=150, validation_data = (xtest_pad, ytest))

Epoch 1/20
459/459 [==============================] - 99s 194ms/step - loss: 0.4216 - accuracy: 0.8075 - val_loss: 0.4008 - val_accuracy: 0.8245
Epoch 2/20
459/459 [==============================] - 67s 147ms/step - loss: 0.3860 - accuracy: 0.8295 - val_loss: 0.3657 - val_accuracy: 0.8331
Epoch 3/20
459/459 [==============================] - 66s 143ms/step - loss: 0.3722 - accuracy: 0.8346 - val_loss: 0.3699 - val_accuracy: 0.8259
Epoch 4/20
459/459 [==============================] - 66s 144ms/step - loss: 0.3636 - accuracy: 0.8402 - val_loss: 0.4787 - val_accuracy: 0.7468
Epoch 5/20
459/459 [==============================] - 65s 141ms/step - loss: 0.3590 - accuracy: 0.8404 - val_loss: 0.3714 - val_accuracy: 0.8388
Epoch 6/20
459/459 [==============================] - 64s 140ms/step - loss: 0.3493 - accuracy: 0.8468 - val_loss: 0.3836 - val_accuracy: 0.8360
Epoch 7/20
459/459 [==============================] - 65s 141ms/step - loss: 0.3457 - accuracy: 0.8478 - val_loss: 0.3780 - val_ac

In [ ]:
ypred_train = (model.predict(xtrain_pad) > 0.5).astype(int)
print(classification_report(ytrain, ypred_train))

2150/2150 [==============================] - 40s 18ms/step
              precision    recall  f1-score   support

         0.0       0.92      0.92      0.92     34576
         1.0       0.92      0.92      0.92     34209

    accuracy                           0.92     68785
   macro avg       0.92      0.92      0.92     68785
weighted avg       0.92      0.92      0.92     68785



In [ ]:
ypred = (model.predict(xtest_pad) > 0.5).astype(int)
print(classification_report(ytest, ypred))

22/22 [==============================] - 1s 25ms/step
              precision    recall  f1-score   support

         0.0       0.82      0.81      0.81       340
         1.0       0.82      0.83      0.82       355

    accuracy                           0.82       695
   macro avg       0.82      0.82      0.82       695
weighted avg       0.82      0.82      0.82       695

